# Q1. Install the package
To get started with MLflow you'll need to install the appropriate Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?

In [1]:
!mlflow --version

mlflow, version 2.1.1


# Q2. Download and preprocess the data
We'll use the Green Taxi Trip Records dataset to predict the amount of tips for each trip.

Download the data for January, February and March 2022 in parquet format from here.

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will:

* load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),

* fit a DictVectorizer on the training set (January 2022 data),

* save the preprocessed datasets and the DictVectorizer to disk.

Your task is to download the datasets and then execute this command:

In [2]:
!python3 preprocess_data.py --raw_data_path data/ --dest_path ./output

So what's the size of the saved DictVectorizer file?

* 54 kB

* --> **154 kB** <--

* 54 MB

* 154 MB

# Q3. Train a model with autolog
We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework.

The script will:

* load the datasets produced by the previous step,

* train the model on the training set,

* calculate the RMSE score on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

In [4]:
!python3 train.py

2023/05/29 11:31:01 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/05/29 11:31:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/unaigaraymaestre/miniconda/envs/mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
RMSE: 2.453983836538874


What is the value of the max_depth parameter:

* 4

* 6

* 8

* --> **10** <--

## Launch the tracking server locally for MLflow
Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

In case of MLflow, you need to:

* launch the tracking server on your local machine,

* select a SQLite db for the backend store and a folder called artifacts for the artifacts store.

You should keep the tracking server running to work on the next three exercises that use the server:

mlflow ui --backend-store-uri sqlite:///mlflow.db -p 5001

# Q4. Tune model hyperparameters
Now let's try to reduce the validation error by tuning the hyperparameters of the RandomForestRegressor using optuna. We have prepared the script hpo.py for this exercise.

Your task is to modify the script hpo.py and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the objective function during the optimization,

* the RMSE obtained on the validation set (February 2022 data).

In [5]:
!python3 hpo.py

[I 2023-05-29 12:12:46,325] A new study created in memory with name: no-name-b3227f12-b376-4a18-babc-9393f656c758
[I 2023-05-29 12:12:46,831] Trial 0 finished with value: 2.451379690825458 and parameters: {'n_estimators': 25, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 2.451379690825458.
[I 2023-05-29 12:12:46,921] Trial 1 finished with value: 2.4667366020368333 and parameters: {'n_estimators': 16, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 2.451379690825458.
[I 2023-05-29 12:12:47,417] Trial 2 finished with value: 2.449827329704216 and parameters: {'n_estimators': 34, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 2 with value: 2.449827329704216.
[I 2023-05-29 12:12:47,603] Trial 3 finished with value: 2.460983516558473 and parameters: {'n_estimators': 44, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 1}. Best is trial 2 with value: 2.44982732

What's the best validation RMSE that you got?

* 1.85

* 2.15

* --> **2.45** <--

* 2.85

# Q5. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called register_model.py, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2022 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tips for MLflow:

* you can use the method search_runs from the MlflowClient to get the model with the lowest RMSE,

* to register the model you can use the method mlflow.register_model and you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", and the name of the model (make sure to choose a good one!).

In [10]:
!python3 register_model.py

2023/05/29 12:43:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/unaigaraymaestre/miniconda/envs/mlops/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Registered model 'random-forest-hyperopt-taxi-model' already exists. Creating a new version of this model...
2023/05/29 12:43:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-hyperopt-taxi-model, version 2
Created version '2' of model 'random-forest-hyperopt-taxi-model'.


What is the test RMSE of the best model?

* 1.885

* 2.185

* 2.555

* 2.955